In [ ]:
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt

# Parameters
fs = 44100          # Sampling rate
duration = 2        # Seconds to record
num_trials = 5      # Number of recordings to average

recordings = []

print("Get ready to clap or pop a balloon...")
for i in range(num_trials):
    input(f"Press ENTER and make the sound (Trial {i+1})")
    
    recording = sd.rec(int(duration * fs),
                        samplerate=fs,
                        channels=1,
                        dtype='float64')
    sd.wait()
    recordings.append(recording[:, 0])

# Convert to array
recordings = np.array(recordings)

# Averaging
averaged_ir = np.mean(recordings, axis=0)

# Normalize
averaged_ir /= np.max(np.abs(averaged_ir))

# Time axis
t = np.linspace(0, duration, len(averaged_ir))

# Plot
plt.figure()
plt.plot(t, averaged_ir)
plt.title("Averaged Room Impulse Response")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.grid()
plt.show()
